In [ ]:
# to test whether object identity is preserved during pickling...

import pickle
import json


class A:
    def __init__(self, x):
        self.x = x


class B:
    def __init__(self, y, a: A):
        self.y = y
        self.a = a


a1 = A(1)
a2 = A(2)
b1 = B(100, a1)
b2 = B(200, a1)
b3 = B(300, a2)
b4 = B(400, a2)
c1 = [a1, a2, b1, b2, b3, b4]
foo = lambda x, i: [x[i] == a for a in [x[0], x[1], x[2].a, x[3].a, x[4].a, x[5].a]]
d1 = foo(c1, 0)
d2 = foo(c1, 1)
data = pickle.dumps(c1)
c2 = pickle.loads(data)
e1 = foo(c2, 0)
e2 = foo(c2, 1)
for x in [d1, e1, d2, e2]:
    print(x)
print(d1 == e1 and d2 == e2)

In [ ]:
from netifaces import interfaces, ifaddresses, AF_INET

for ifaceName in interfaces():
    addresses = [
        i["addr"]
        for i in ifaddresses(ifaceName).setdefault(AF_INET, [{"addr": "No IP addr"}])
    ]
    print("%s: %s" % (ifaceName, ", ".join(addresses)))

In [ ]:
from netifaces import interfaces, ifaddresses, AF_INET

print(interfaces())
print(ifaddresses("en0").setdefault(AF_INET))
print(ifaddresses("en0").setdefault(AF_INET)[0]["addr"])

In [ ]:
class A:
    def __init__(self):
        pass


class B:
    def __init__(self):
        pass


class C:
    def __init__(self):
        pass


def u(a, b=None):
    return Union[a, b] if b else a


def s_u(lis):
    r = Union[lis[0], lis[1]]
    r = Union[r, lis[2]]
    return r


print("s_u", s_u([A, B, C]))


def s_u_2(lis):
    r = None
    if len(lis) >= 1:
        for c in lis:
            r = Union[r, c]
    return r


print("s_u_2", s_u_2([]))
print("s_u_2", s_u_2([A]))
print("s_u_2", s_u_2([A, B]))
print("s_u_2", s_u_2([A, B, C]))

un = lambda a, b=None: Union[a, b] if b else a

print((lambda a, b: a + b)(1, 2))
print(Union[A, B])
print(Union[A, Union[B]])

print(list(u(x for x in base.Action.__subclasses__())))
print(
    list(
        (lambda a, b=None: Union[a, b] if b else a)(
            x for x in base.Action.__subclasses__()
        )
    )
)


def subclass_union(Class):
    subclasses = Class.__subclasses__()
    binary_union = lambda a, b=None: Union[a, b] if b else a
    return binary_union(x for x in subclasses)


print(subclass_union(base.Action))

In [ ]:
from tempfile import NamedTemporaryFile, TemporaryDirectory

t = NamedTemporaryFile()
try:
    print(t.name)
except:
    print("invalid file method name")
t.close()
d = TemporaryDirectory()
try:
    print(d.name)
except:
    print("invalid directory method name")
d.cleanup()

In [ ]:
from tempfile import gettempdir

gettempdir()

In [ ]:
from whendo.core.action import Action

list(s for s in Action.__subclasses__())

In [ ]:
import pathlib

str(pathlib.Path().home())

In [ ]:
import os

[
    dirpath
    for dirpath, dirnames, filenames in os.walk("whendo")
    if os.path.exists(os.path.join(dirpath, "__init__.py"))
]

In [ ]:
import psutil

In [ ]:
psutil.virtual_memory()._asdict()

In [ ]:
psutil.net_if_addrs()["en0"][0].address

In [ ]:
psutil.cpu_percent()

In [ ]:
dict(zip(["1min", "5min", "15min"], psutil.getloadavg()))

In [ ]:
psutil.virtual_memory()._fields

In [ ]:
dict(
    zip(
        psutil.virtual_memory()._fields,
        ["{:,}".format(n) for n in psutil.virtual_memory()],
    )
)

In [ ]:
import whendo.core.actions.http_action as http_x
import whendo.core.schedulers.timed_scheduler as sched_x
import whendo.core.actions.file_action as file_x
import whendo.core.actions.logic_action as logic_x
import whendo.core.actions.sys_action as sys_x
from whendo.sdk.client import Client

client = Client(host="192.168.0.26")
client.clear_dispatcher()

In [ ]:
file_append = file_x.FileAppend(file="executeaction.txt", payload={"a": "b"})
execute_action = http_x.ExecuteAction(
    host="192.168.0.26", port=8000, action_name="file_append"
)
client.add_action("file_append", file_append)
client.add_action("execute_action", execute_action)

In [ ]:
execute_action.execute()

In [ ]:
client.execute_action("execute_action")

In [ ]:
class Compose:
    def __init__(self, actions):
        self.actions = actions

    def execute(self, tag: str = None, data: dict = None):
        for action in self.actions:
            data = action.execute(tag=tag, data=data)
        return data

In [ ]:
import whendo.core.actions.logic_action as logic_x
from whendo.core.action import Action


class FleaCount(Action):
    flea_count: int = 0

    def execute(self, tag: str = None, data: dict = None):
        self.flea_count += 1


fc = FleaCount()
c = logic_x.All(actions=[fc, fc])
c.execute()

In [ ]:
class C:
    def __init__(self, c: int):
        self.c = c

    def foo(self):
        self.c += 1


x = C(c=1)


def bar(y):
    y.foo()


bar(x)
x.__dict__

In [ ]:
import whendo.core.actions.logic_action as logic_x
c = logic_x.compose(actions[])

In [ ]:
from collections import namedtuple

action_result = namedtuple(
    "action_result", ["result", "data", "extra"], defaults=(None,) * 3
)

action_result(1, 2, None)

In [ ]:
(None,) * 2

In [ ]:
from dataclasses import dataclass